In [22]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\code") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile(r'D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\data\jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

# (corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [23]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) =load_data_jay_lyrics()

In [35]:
#define model
num_hiddens=256
#rnn_layer=nn.LSTM(input_size=vocab_size,hidden_size=num_hiddens)
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

In [36]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
Y,state_new=rnn_layer(X,state)
print(Y.shape,len(state_new),state_new[0].shape)#输出形状为（时间步数，批量大小，隐藏单元个数），
                                                #隐藏状态的形状为（层数，批量大小，隐藏单元个数）

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [43]:
#define RNN model with an dense output
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModel,self).__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size*(2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        self.state=None
        
    def forward(self,inputs,state):
        #获取one-hot向量
        X=d2l.to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        #全连接层会将y的形状变成（num_steps*batch_size,num_hiddens）
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state

In [61]:
#predict rnn model based on the predix by the largest weight
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]#output 会记录prefix加上输出
    for t in range(num_chars+len(prefix)-1):
        X=torch.tensor([output[-1]],device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple):#LSTM,state：（h,c）
                state=(state[0].to(device),state[1].to(device))
            else:
                state=state.to(device)
                
        (Y,state)=model(X,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [62]:
model=RNNModel(rnn_layer,vocab_size).to(device)
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开渡靠条条条条条条条条'

In [71]:
#train model
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,
                                 num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
    loss=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum,n,start=0.0,0,time.time()
        data_iter=d2l.data_iter_consecutive(corpus_indices,batch_size,num_steps,device)#相邻采样
        for X,Y in data_iter:
            if state is not None:
                #使用detach函数从计算图中分离隐藏态，这是为了
                #使得模型参数的梯度计算只依赖一次迭代读取的小批量序列（防止梯度开销太大）
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
            (output,state)=model(X,state)#output 形状为（num_steps*batch_size,vocab_size）
            
            #Y的形状是(batch_size,num_steps)，转置后便哼长度batch_size*num_steps的向量，跟输出行一一对应
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(output,y.long())
            
            optimizer.zero_grad()
            l.backward()
            #梯度裁剪
            d2l.grad_clipping(model.parameters(),clipping_theta,device)
            optimizer.step()
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
        
        try:
            perplexity=math.exp(l_sum/n)
        except OverflowError:
            perplexity=float('inf')
        if (epoch+1)% pred_period==0:
            print("epoch %d, perplexity %f,time %.2f sec" % (
            epoch+1,perplexity,time.time()-start))
            
            for prefix in prefixes:
                print('-',predict_rnn_pytorch(
                prefix,pred_len,model,vocab_size,device,idx_to_char,char_to_idx))

In [72]:
num_epoches,batch_size,lr,clipping_theta=250,32,1e-3,1e-2
pred_period,pred_len,prefixes= 50,50,["分开","不分开"]
train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,
                             char_to_idx,num_epoches,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 50, perplexity 10.396823,time 0.95 sec
- 分开始我不 再想 你不的我  你想要你 我不多 我 你不我 你你在我想你 我想你 我不多 别人我  你是
- 不分开 我想要你 我想多 我爱你 我想多 我不能 我想你你的爱我 你这样的手不  想要你你 我有了这样我想
epoch 100, perplexity 1.297595,time 0.96 sec
- 分开 我有人难过 还在空 快谁是神不了 习惯 在吸完血后开始打呼 管家是一只会说法语举止优雅的猪 吸血前
- 不分开 我我不能痛  我不 我想你 不要去  回小的外桌椅 一起看着  想带和 看着的河爱 说 别你的世爸
epoch 150, perplexity 1.067417,time 0.96 sec
- 分开 我有人难过 还已经离你我 我有多难熬  穿过云层 我试著努力向你奔跑 爱才送到 你却已在别人怀抱 
- 不分开 了我来不多   没有你在我不能再这样 我们就很我不多 样 我不想你想要你我的手不能 语你可 看你已
epoch 200, perplexity 1.034511,time 0.98 sec
- 分开 我有人练 你 一壶好酒 再来一碗热粥 配上几斤的牛肉 我说店小二 三两银够不够 景色入秋 漫天黄沙
- 不分开 我我不好多  没有没有你说 有是难过  不懂 你 一定有痛 像龙的可爱 我 想这样的甜蜜 所有一双
epoch 250, perplexity 1.023160,time 0.98 sec
- 分开 我有人练 多 已经猜透 透不想通你 我的脚起 戒指在空屋 在灌木丛旁邂逅 一只令它心仪的母斑鸠 印
- 不分开 我我不好多  多说没多 说是我想要你看棒  想要这 我已无处可躲 我不要再想 我不要再想 我不 我
